In [ ]:
#完成程式-前後景去除
from keras.models import load_model
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

model = load_model('model_finger_2.h6')

cap_region_x_begin = 0.5  
cap_region_y_end = 0.8
threshold = 60  
blurValue = 41  
bgSubThreshold = 100
learningRate = 0

isBgCaptured = 0
triggerSwitch = False


def printThreshold(thr):
    print("! Changed threshold to " + str(thr))


def removeBG(frame):
    fgmask = bgModel.apply(frame, learningRate=learningRate)
    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    return res

camera = cv2.VideoCapture(0)   
camera.set(10, 200)  
cv2.namedWindow('trackbar') 
cv2.resizeWindow("trackbar", 640, 200)  
cv2.createTrackbar('threshold', 'trackbar', threshold, 100, printThreshold)

while camera.isOpened():
    ret, frame = camera.read()
    threshold = cv2.getTrackbarPos('threshold', 'trackbar') 
    frame = cv2.bilateralFilter(frame, 5, 50, 100) 
    frame = cv2.flip(frame, 1) 
    cv2.rectangle(frame, (int(cap_region_x_begin * frame.shape[1]), 0),(frame.shape[1], int(cap_region_y_end * frame.shape[0])), (0, 0, 255), 2)
    
    cv2.putText(frame," ",(0,40),cv2.FONT_HERSHEY_COMPLEX,1,(0, 255, 0),1)

    if isBgCaptured == 1:
        img = removeBG(frame) 
        img = img[0:int(cap_region_y_end * frame.shape[0]),int(cap_region_x_begin * frame.shape[1]):frame.shape[1]] 

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0) 
        
        ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)  

                IMG_SIZE = 128

        thresh = cv2.resize(thresh, (IMG_SIZE, IMG_SIZE)).tolist()
        thresh = np.array(thresh, dtype='int8').reshape(-1, IMG_SIZE, IMG_SIZE, 1)

        CATEGORIES = ["NONE", "ONE", "TWO", "THREE", "FOUR", "FIVE"]
        pred = CATEGORIES[np.argmax(model.predict(thresh))]

        cv2.putText(frame,str(pred),(0,40),cv2.FONT_HERSHEY_COMPLEX,1,(0, 255, 0),1)
        
    cv2.imshow('original', frame)

    k = cv2.waitKey(10)
    if k == 27:
        break
    elif k == ord('b'):
        bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold,False)
        isBgCaptured = 1
        print('in')